# Segment Anything Model for Geospatial Data

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/satellite.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)

This notebook shows how to use segment satellite imagery using the Segment Anything Model (SAM) with a few lines of code.

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.


In [10]:
pip install segment-geospatial leafmap localtileserver

  Using cached segment_geospatial-0.8.5-py2.py3-none-any.whl (37 kB)
  Using cached leafmap-0.23.0-py2.py3-none-any.whl (1.8 MB)
  Using cached localtileserver-0.7.1-py3-none-any.whl (12.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 716.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 114.6 MB/s eta 0:00:00
     

## Import libraries

In [11]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

## Create an interactive map

In [12]:
# m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
m = leafmap.Map(center=[31.079004, -97.608278], zoom=19) #Nolanville TX
m.add_basemap("SATELLITE")


Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [14]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    # bbox = [-95.3704, 29.6762, -95.368, 29.6775]
    bbox = [-97.57868441311106, 31.052219092962858, -97.65101491463611, 31.106173615520717]

## Download map tiles

Download maps tiles and mosaic them into a single GeoTIFF file

In [15]:
image = "satellite.tif"

Besides the `satellite` basemap, you can use any of the following basemaps returned by the `get_basemaps()` function:

In [16]:
# get_basemaps().keys()

Specify the basemap as the source.

In [ ]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Streaming output truncated to the last 5000 lines.
Downloaded image 34222/39220
Downloaded image 34223/39220
Downloaded image 34224/39220
Downloaded image 34225/39220
Downloaded image 34226/39220
Downloaded image 34227/39220
Downloaded image 34228/39220
Downloaded image 34229/39220
Downloaded image 34230/39220
Downloaded image 34231/39220
Downloaded image 34232/39220
Downloaded image 34233/39220
Downloaded image 34234/39220
Downloaded image 34235/39220
Downloaded image 34236/39220
Downloaded image 34237/39220
Downloaded image 34238/39220
Downloaded image 34239/39220
Downloaded image 34240/39220
Downloaded image 34241/39220
Downloaded image 34242/39220
Downloaded image 34243/39220
Downloaded image 34244/39220
Downloaded image 34245/39220
Downloaded image 34246/39220
Downloaded image 34247/39220
Downloaded image 34248/39220
Downloaded image 34249/39220
Downloaded image 34250/39220
Downloaded image 34251/39220
Downloaded image 34252/39220
Downloaded image 34253/39220
Downloaded image 3425

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [ ]:
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

![](https://i.imgur.com/KAm84IY.png)

## Initialize SAM class

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

## Segment the image

Set `batch=True` to segment the image in batches. This is useful for large images that cannot fit in memory.

In [ ]:
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

## Polygonize the raster data

Save the segmentation results as a GeoPackage file.

In [ ]:
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

You can also save the segmentation results as any vector data format supported by GeoPandas.

In [ ]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

## Visualize the results

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m

![](https://i.imgur.com/Ysq3u7E.png)